# Brightway2 seminar
Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

## Day 1, afternoon
### Session on uncertainty in Brightway

### Learning objectives  
  - Learn how uncertainty is represented in exchanges  
  - Learn how to carry out a MonteCarloLCA  

Standard inputs

In [5]:
import brightway2 as bw
import numpy as np

Open the seminar project:

In [2]:
bw.projects.set_current('bw2_seminar_2017')

### 1) Uncertainty information in Brightway exchanges

Uncertainty is stored at the level of the exchanges.  

**Sample exchange from ecoinvent v3.3 (`lognormal`**)   
Let's look at a random exchange from ecoinvent 3.3, from which we removed some fields that were not necessary for the purpose of discussing uncertainty :

In [3]:
some_exc_from_ecoinvent = {'name': 'ethylene, average',
                           'type': 'technosphere',
                           'amount': 0.23244,
                           'unit': 'kilogram',
                           'input': ('ecoinvent 3.3 cutoff', '14db59eea64e46a1e8332973f714a3ef'),
                           'output': ('ecoinvent 3.3 cutoff', 'df875d1e65cd48bc1ac69b960e172e85'),
                           'uncertainty type': 2,
                           'loc': -1.459123151775232,
                           'scale without pedigree': 0.1414213562373095,
                           'scale': 0.2,
                           'pedigree': {
                               'completeness': 5,
                               'further technological correlation': 1,
                               'geographical correlation': 5,
                               'reliability': 4,
                               'temporal correlation': 3
                           }
                          }

In this *specific* case, the *necessary* uncertainty of the exchange are described in the following fields:  
  - **'uncertainty type'** : type of probability distribution function that the exchange follows. In this case, the exchange has an uncertainty type = 2, indicating it is a `lognormal`  distribution.  
  - **'loc', 'scale'**: parameters of the lognormal distribution function, which are respectively the mean $\mu$ and the standard deviation $\sigma$ of the underlying normal distribution (more on this later).  
  
Other probability distributions functions (e.g. normal, triangular, etc.) can also be modelled, and will each have their own specific paramters (more on this later). 

Some *additional* uncertainty related information ('scale without pedigree', 'pedigree') are also there, but are not directly used in the calculation of the uncertainty. They are also specific to ecoinvent. 

Uncertainty in Brightway is dealt with using a Python package called `stats_arrays` (see [here](http://stats-arrays.readthedocs.io/en/latest/)), developed by Chris Mutel in the context of the development of Brightway but applicable to any stochastic model in Python. Let's import it. 

In [4]:
import stats_arrays

Before delving into the details of ` stats_arrays`, let's have a quick look at the uncertainty information above. 
As a reminder:   
  - a random variable $X$ is a lognormal if its natural logarithm $ln(X)$ is normally distributed  
  - the natural logarithm of the *median* of the lognormal distribution is equal to the median (=mean) of the underlying distribution  

Taking the deterministic amount `amount` to be the median, we should have `loc` = `ln('amount')`. 

In [9]:
some_exc_from_ecoinvent['loc'] == np.log(some_exc_from_ecoinvent['amount'])

True

The link between the geometric standard deviation ($GSD$) of $X$ and the standard deviation of $ln(X)$ ($/mu$) is:  $GSD=e^{\mu}$. The $GSD^2$ og the sample exchange above is therefore:

In [11]:
GSD2 = np.exp(some_exc_from_ecoinvent['scale'])**2
GSD2

1.4918246976412703

Approximately 95% of the values for the lognormally distributed parameter $X$ are between $median/GSD^2$ and $median*GSD^2$. In the example:  

In [18]:
print("Approx. 95% of the values are in the range [{}, {}]".format(some_exc_from_ecoinvent['amount']/GSD2, some_exc_from_ecoinvent['amount']*GSD2))

Approx. 95% of the values are in the range [0.155809191500524, 0.34675973271973687]


In a Monte Carlo simulation, we would want to sample a large number of values for many random variables and recalculate results using these randomly sampled values. This is where `stats_arrays` comes in.  

The `stats_arrays` methods requires parameter arrays for the random variables. These parameter arrays contain exactly seven paramters per random variable, and are stored as a Numpy structured array. These seven parameters are:  
 - 'loc' (first shape parameter of the distribution, related to the central tendency  
 - 'scale' (second shape parameter, related to the dispersion of the paramter)  
 - 'shape' (third shape parameter, related to the dispersion's skewness)  
 - 'minimum' (lower limit to the values that can be sampled. This can be a core parameter of the distribution (e.g. for the uniform distribution) of an optional parameter (e.g. for normal distributions for which negatove values are excluded).  
 - 'maximum' (upper limit to the values that can be sampled - see 'minimum'.  
 - 'negative' (an indication of whether the random variable is negative. useful for generating e.g. "negative lognormals".
 - 'uncertainty_type' Type of uncertainty (normal, lognormal, triangular, etc.)  

The following table, taken from [here](http://stats-arrays.readthedocs.io/en/latest/), that states which paramters are required, which are optional and which are not relevant for the distribution functions explicitly modeled now in `stats_arrays`  
<img src="images/stats_arrays_table.JPG">

One can create a stats array using the class method `stats_arrays.UncertaintyBase.from_dicts' 

In [26]:
# Creating a stats array for three random variables. 
my_uncertain_variables = stats_arrays.UncertaintyBase.from_dicts(
    {'loc': some_exc_from_ecoinvent['loc'], 
     'scale': some_exc_from_ecoinvent['scale'], 
     'uncertainty_type': some_exc_from_ecoinvent['uncertainty type']},
    {'loc': 2, 'scale': 0.5, 'uncertainty_type': stats_arrays.NormalUncertainty.id},
    {'loc': 1.5, 'minimum': 0, 'maximum': 10, 'uncertainty_type': stats_arrays.TriangularUncertainty.id}
)
my_uncertain_variables

array([(-1.45912315,  0.2,  nan,  nan,  nan, False, 2),
       ( 2.        ,  0.5,  nan,  nan,  nan, False, 3),
       ( 1.5       ,  nan,  nan,   0.,  10., False, 5)], 
      dtype=[('loc', '<f8'), ('scale', '<f8'), ('shape', '<f8'), ('minimum', '<f8'), ('maximum', '<f8'), ('negative', '?'), ('uncertainty_type', 'u1')])

One can then use the `stats_arrays.MCRandomNumberGenerator` to generate samples for the differnt variables, which is a generator for which we can use the `next` command:

In [48]:
my_rng = stats_arrays.MCRandomNumberGenerator(my_uncertain_variables)

In [49]:
# Using `next`:
next(my_rng)

array([ 0.29666699,  1.16691997,  4.42700975])

In [50]:
# Using `next` in a loop:
np.array([rng.next() for _ in range(10)])

array([[ 0.25736467,  2.19696566,  1.16556844],
       [ 0.26461269,  2.66602054,  3.99102199],
       [ 0.27751033,  1.29639883,  3.94880135],
       [ 0.28552087,  2.50509276,  3.10439448],
       [ 0.19507183,  2.04155515,  4.127155  ],
       [ 0.17594805,  1.71756024,  5.99113623],
       [ 0.19115472,  1.56258351,  5.69921973],
       [ 0.28000925,  1.83493566,  2.22506036],
       [ 0.28510989,  2.1160002 ,  7.77394375],
       [ 0.28510989,  2.1160002 ,  7.77394375]])

## 2) Monte Carlo calculations in Brightway

### General syntax

In [52]:
random_act = bw.Database('ecoinvent 2.2').random()
random_act

'soybean methyl ester, production US, at service station' (kilogram, CH, ['biomass', 'fuels'])

In [54]:
myFirstMonteCarlo = bw.MonteCarloLCA({random_act:1},  ('IPCC 2013', 'climate change', 'GWP 100a'))
scores = [next(myFirstMonteCarlo) for _ in range(10)]
scores

[1.1706241632075627,
 1.3682146388006031,
 1.5845670676861614,
 1.3320895426544892,
 1.1765512430486047,
 1.366758878086722,
 1.361414611405248,
 1.2099967961537306,
 1.4448611177777781,
 1.2664083790827274]

### How it works  
  - Matrices are built as in standard LCA (using the approach seen this morning `builder`)  
  - The `MCRandomNumberGenerator` generates new values for all paramters in the $A$ and $B$ matrices (and for characterization factors if the LCA has an LCIA component, i.e. if `MonteCarloLCA` was instantiatied with a method).  
  - For every subsequent iteration, matrices are rebuilt with new sampled values BUT same indices, cutting down on matrix building time  
  - Rather than solving the system of linear equations directly, an *iterative solver* is used. This takes as an initial guess the solution found with the paramter values from the first Monte Carlo iteration.  
  ## - CHRIS PLEASE CONFIRM THE FORMER THE DEFAULT IN THE MonteCarloLCA - I thought so, but can't tell from the source code.
  

### Monte Carlo using multiple impact assessment methods (no uncertainty on characterization factors)
There is presently no method in Brightway that does MonteCarlo iterationswith multiple LCIA methods. This can be annoying since the results obtained for different impact categories are correlated (they depend on some of the same paramters from the $A$ and, to a smaller extent, $B$ matrix), but this correlation is lost when you need to do Monte Carlo simulations sequentially. 
There are two obvious solutions (and probably many other, not so obvious solutions):  

**Solution 1**: One can store the $A$ and $B$ matrix values and reuse them. This is possible using the `ParameterVectorLCA` class of Monte Carlo, which generates a single array with all these parameters as well as sampled characterization factors. While not implemented, it would be possible to concatenate many such vectors (one per MonteCarlo iteration) and recalculate LCIA scores using matrices built using `ParameterVectorLCA.rebuild_all`.

**Solution 2**: If there is no uncertainy associated with the characterization factors, one can instead store characterization matrices and multiply the inventory resulting from MonteCarloLCA for each iteration.  

**Exercise**: Write a function that uses MonteCarloLCA and calculates arrays of scores for multiple impact categories. Test your function with a functional unit = {random_act:1}, list of methods = all ILCD methods (with long term emissions) and 1000 MOnte Carlo iterations.

Step 1) Create a MonteCarloLCA object with some functional unit but no method. Run `.lci` to build the $A$ and $B$ matrices (and hence fix the indices of our matrices)  
Step 2) Create a 'C_matrices' dictionary (empty for now), that will collect characterization factor matrices (C matrices). Keys will be the tuple representing the method name, and the values will be the actual matrices.  
Step 3) Write a `for` loop that iterates over a list of method names and stores the method_name:C_matrix in the dictionary. Use `switch_method` to do this.  
Once this dictionary is built:  
Step 4): Create an empty array (`np.empty`) of dimension equal to the number of methods that will be considered (rows) by the number of iterations required (columns).  
Step 5) Populate this array using a `for` loop over the number of MonteCarlo iterations required (use `next` to rebuild the $A$ and $B$ matrices). Include a nested `for` loop over the methods and multiply the characterization matrix with the inventory.  
Step 6) Take all this code and structure it in a function that takes as arguments (1) the functional unit, (2) a list of method names and (3) the number of iterations.

In [68]:
def multiImpactMonteCarloLCA(functional_unit, list_methods, iterations):
    # Step 1
    MC_lca = bw.MonteCarloLCA(functional_unit)
    MC_lca.lci()
    # Step 2
    C_matrices = {}
    # Step 3
    for method in list_methods:
        MC_lca.switch_method(method)
        C_matrices[method] = MC_lca.characterization_matrix
    # Step 4
    results = np.empty((len(list_methods), iterations))
    # Step 5
    for iteration in range(iterations):
        next(MC_lca)
        for method_index, method in enumerate(list_methods):
            results[method_index, iteration] = (C_matrices[method]*MC_lca.inventory).sum()
    return results

In [62]:
ILCD = [method for method in bw.methods if "ILCD" in str(method) and "no LT" not in str(method)]
fu = {random_act:1}
iterations = 100

In [69]:
test_results = multiImpactMonteCarloLCA(fu, ILCD, iterations)

In [70]:
test_results

array([[  1.13342404e+00,   1.18007044e+00,   1.48454903e+00, ...,
          1.11038821e+00,   1.42487355e+00,   1.28508097e+00],
       [  1.02924251e-03,   9.72600621e-04,   1.04048818e-03, ...,
          8.77505680e-04,   1.46729939e-03,   1.13511586e-03],
       [  5.80484929e+00,   1.19983961e+01,   5.26334387e+00, ...,
          2.74226221e+00,   8.41831101e+00,   1.55299517e+01],
       ..., 
       [  8.55066182e-08,   1.10786040e-07,   1.28906601e-07, ...,
          1.06863289e-07,   1.21667911e-07,   1.45760330e-07],
       [  3.91987641e-07,   2.50497337e-07,   2.45303470e-07, ...,
          2.20911414e-07,   3.72427451e-07,   5.41403488e-07],
       [  1.18968842e-02,   1.27351605e-02,   1.37002713e-02, ...,
          1.07569110e-02,   1.24859416e-02,   2.93620049e-02]])